In [1]:
import awswrangler as wr

import mlflow

# Para que funciones, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


# Entrenamos el modelo con LDA

Probaremos el modelo con LDA, y a su vez, el dataset nos servirá como base para otros modelos.

In [2]:
mlflow_server = "http://localhost:5001"
mlflow.set_tracking_uri(mlflow_server)
s3_base_path = "s3://data/chicago/crimes/2024"

In [3]:
# Cargamos los datos para realizar nuestro estudio.
X_train =  wr.s3.read_csv(f"{s3_base_path}/final/X_train.csv")
y_train =  wr.s3.read_csv(f"{s3_base_path}/final/y_train.csv")

X_test =  wr.s3.read_csv(f"{s3_base_path}/final/X_test.csv")
y_test =  wr.s3.read_csv(f"{s3_base_path}/final/y_test.csv")

## Arrancamos a experimentar

In [4]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def metrics(model, X_test, y_test):

    y_pred = model.predict(X_test)
    cr = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

    # Graficamos matriz de confusion
    categories = y_test["fbi_code"].astype("category").cat.categories
    cm = confusion_matrix(y_test, y_pred, labels=categories)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=categories)
    fig, ax = plt.subplots(figsize=(15,15))
    ax.grid(False)
    disp.plot(ax=ax)
    plt.close(fig)

    return cr, fig

Antes de poder realizar experimentos, vamos a crear el experimento en MLFLow, pero para evitar desorden, vamos a usar una función que se fije primero si el experimento existe, si esto es así, devuelve su ID.

Además creamos el nombre del run padre con el que vamos a ir registrando las ejecuciones.

In [5]:
import datetime
from mlflow_aux import get_or_create_experiment
# Creemos el experimento
experiment_id = get_or_create_experiment("Chicago Crimes 2024")
print(experiment_id)
run_name_parent = "lda_test"  + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')

1


In [6]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mlflow.models import infer_signature

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):
    lda = LinearDiscriminantAnalysis(n_components=2)
    X_train_lda = lda.fit_transform(X_train, y_train["fbi_code"])
    # Aplicamos a los datos de test
    X_test_lda = lda.transform(X_test)
    cr, cm = metrics(lda, X_test, y_test)

    # Una vez que terminamos la búsqueda, guardamos los mejores parámetros en el run padre.
    mlflow.log_params(lda.get_params())
    # Logging all metrics in classification_report
    mlflow.log_metric("test_accuracy", cr.pop("accuracy"))
    for class_or_avg, metrics_dict in cr.items():
        for metric, value in metrics_dict.items():
            mlflow.log_metric("test_" + class_or_avg + '_' + metric,value)
    mlflow.log_figure(figure=cm, artifact_file="fda_cm.png")

    # Guardamos el artefacto del modelo
    artifact_path = "model"

    signature = infer_signature(X_train, lda.predict(X_train))

    mlflow.sklearn.log_model(
        sk_model=lda,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format='cloudpickle',
        registered_model_name="lda_model_dev",
        metadata={"model_data_version": 1}
    )

    # Obtenemos la ubicación del modelo guardado en MLFlow
    model_uri = mlflow.get_artifact_uri(artifact_path)


2025/08/05 02:29:23 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'lda_model_dev'.
2025/08/05 02:29:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: lda_model_dev, version 1
Created version '1' of model 'lda_model_dev'.


🏃 View run lda_test2025/08/05-02:29:15" at: http://localhost:5001/#/experiments/1/runs/65110dd56bd14282b4c8d2c810b5f5ea
🧪 View experiment at: http://localhost:5001/#/experiments/1


## Registramos el modelo 

Realizamos el registro del modelo en MLflow. En este registro se pone el modelo productivo que luego se usará para servir en formato on-line.

In [7]:
from mlflow import MlflowClient

client = MlflowClient()
name = "lda_model_prod"
desc = "This classifier show FBI code based on Chicago Police Report"

# Creamos el modelo productivo
try:
    client.create_registered_model(name=name, description=desc)
except:
    client.update_registered_model(name=name, description=desc)

# Guardamos como tag los hiper-parametros en la version del modelo
tags = lda.get_params()
tags["model"] = type(lda).__name__
tags["f1-score"] = cr["weighted avg"]["f1-score"]

# Guardamos la version del modelo
result = client.create_model_version(
    name=name,
    source=model_uri,
    run_id=model_uri.split("/")[-3],
    tags=tags
)

# Y creamos como la version con el alias de champion para poder levantarlo en nuestro
# proceso de servicio del modelo on-line.
client.set_registered_model_alias(name, "champion", result.version)

2025/08/05 02:29:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: lda_model_prod, version 1
